In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal
import copy

In [136]:
import time

In [4]:
import streamm

In this getting started example we will calculate the coupling between P3HT oligomers

In [5]:
import logging
logging.basicConfig(filename='p3ht_et.log',level=logging.DEBUG)

Now let's create project and resource to keep track of our work

In [6]:
p3ht_et = streamm.Project('P3HT_ET')

And a resource object to keep track of where our files are 

In [7]:
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [8]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

In [9]:
res_local.make_dir()

In [10]:
p3ht_et.set_resource(res_local)

Now we need to set a remote resource we have ssh access to and run calculations 

In [11]:
peregrine = streamm.Resource('peregrine')

In [12]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = res_local.dir['templates']

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [13]:
pprint(peregrine.properties)

{u'allocation': 'orgopv',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper',
 u'walltime': 4}


In [14]:
peregrine.make_dir()

Explicitely create a thiophene molecule

In [224]:
bbTh = streamm.Buildingblock('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

In [225]:
bbTh.particles[5].rsite = 'termcap'
bbTh.particles[6].rsite = 'funccap'
bbTh.particles[8].rsite = 'termcap'

Set some properties of the molecule to keep track of the parts

In [226]:
c_cnt =1
h_cnt =1

for pkey_i, particle_i  in bbTh.particles.iteritems():

    if( particle_i.symbol == 'C' ):
        particle_i.label = "C%d"%(c_cnt)
        particle_i.resname = "SCP2"
        particle_i.residue = 1

        c_cnt +=1 
    if( particle_i.symbol == 'S' ):
        particle_i.resname = "ThS"
        particle_i.residue = 2

    if( particle_i.symbol == 'H' ):
        particle_i.label = "H%d"%(h_cnt)
        particle_i.resname = "HA"
        particle_i.residue = 3

        h_cnt +=1 


Set the force-field type and guess some reasonable charges 

In [227]:
for pkey_i, particle_i  in bbTh.particles.iteritems():
    if( particle_i.symbol == 'C' ):
        particle_i.paramkey = 'CA'
        particle_i.charge = -0.025
    if( particle_i.symbol == 'S' ):
        particle_i.paramkey = 'S'
        particle_i.charge = -0.3
    if( particle_i.symbol == 'H' ):
        particle_i.paramkey = 'HA'
        particle_i.charge = 0.1

Check molecule is neutral 

In [228]:
total_charge = 0.0
for pkey_i, particle_i  in bbTh.particles.iteritems():
    total_charge += particle_i.charge
print total_charge

-2.77555756156e-17


Optimize structure with NWChem

But let's put it in a function this time

In [20]:
def nw_opt(project_i,bb_i,res_i):
    '''Optimize a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_i = streamm.NWChem('nw_opt_{}'.format(bb_i.tag))
    print nwchem_i.tag 
    # Add thiophene structure 
    nwchem_i.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_i.set_resource(res_i)
    # Make the local directories 
    nwchem_i.make_dir()
    #Change to the `launch` directory
    os.chdir(nwchem_i.dir['launch'])
    # Copy over templates
    nwchem_i.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_i.cp_file('templates','nw',"nwchem.nw",'templates','launch')
    # Read in templates files 
    nwchem_i.load_str('templates','nw')        
    nwchem_i.load_str('templates','run')
    # Set calculation properties 
    nwchem_i.properties['basis'] = '6-31g'
    nwchem_i.properties['method'] = 'UHF'
    nwchem_i.properties['charge'] = 0
    nwchem_i.properties['spin_mult'] = 1
    nwchem_i.properties['task'] = 'SCF optimize'
    nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()
    # 
    pprint(nwchem_i.properties)
    # Replace <key> with properties value 
    nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))
    nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
    nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))
    #
    nwchem_i.add_file('output','log',"%s.log"%(nwchem_i.tag))
    # Save details in .json files 
    os.chdir(nwchem_i.dir['home'])
    p3ht_et.dump_json()
    # 
    os.chdir(nwchem_i.dir['launch'])
    # 
    nwchem_i.push()
    # 
    nwchem_i.run()
    # Add calculation to project
    project_i.add_calc(nwchem_i,deepcopy = True)
    # 
    return project_i 

In [21]:
p3ht_et = nw_opt(p3ht_et,bbTh,peregrine)

nw_opt_thiophene
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.55498576      -1.91131218      -0.00081000 \n     C      -0.17775976      -1.91131218      -0.00081000 \n     C       0.34761524      -0.57904218      -0.00081000 \n     C      -0.65884476       0.36101082       0.00000000 \n     S      -2.16948076      -0.35614618      -0.00000800 \n     H      -2.18966076      -2.79526518      -0.00132100 \n     H       0.45389024      -2.80145418      -0.00106400 \n     H       1.41682424      -0.35961818      -0.00138200 \n     H      -0.51943676       1.44024682       0.00064700 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_opt_thiophene/'

In [22]:
nwchem_i = p3ht_et.calculations['nw_opt_thiophene']

Check status unit finished

In [23]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)    

In [24]:
print nwchem_i.meta['status']

finished


In [25]:
nwchem_i.analysis()

Print energies 

In [26]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.4732391, -0.3475018, -0.3290008, 0.1318747, 0.2130991, 0.2315597, 0.2416684, 0.269473, 0.2970013, 0.3198832]
-551.190758142


Check that the positions of the structure have been optimized 

In [27]:
print bbTh.positions

[[ -1.55498576e+00  -1.91131218e+00  -8.10000000e-04]
 [ -1.77759760e-01  -1.91131218e+00  -8.10000000e-04]
 [  3.47615240e-01  -5.79042180e-01  -8.10000000e-04]
 [ -6.58844760e-01   3.61010820e-01   0.00000000e+00]
 [ -2.16948076e+00  -3.56146180e-01  -8.00000000e-06]
 [ -2.18966076e+00  -2.79526518e+00  -1.32100000e-03]
 [  4.53890240e-01  -2.80145418e+00  -1.06400000e-03]
 [  1.41682424e+00  -3.59618180e-01  -1.38200000e-03]
 [ -5.19436760e-01   1.44024682e+00   6.47000000e-04]]


In [28]:
print nwchem_i.strucC.positions

[[ -1.52900788e+00  -1.96228033e+00  -8.24550000e-04]
 [ -1.87338440e-01  -1.91419006e+00  -1.07736000e-03]
 [  3.42591710e-01  -5.70383860e-01  -6.41090000e-04]
 [ -6.05111100e-01   3.80520380e-01  -3.01900000e-05]
 [ -2.25060957e+00  -3.24146360e-01   4.65300000e-05]
 [ -2.16232414e+00  -2.82023213e+00  -1.02345000e-03]
 [  4.31209940e-01  -2.78809225e+00  -1.53823000e-03]
 [  1.39112655e+00  -3.53910890e-01  -8.21500000e-04]
 [ -4.82375920e-01   1.43982288e+00   3.51820000e-04]]


Update positions with optimized geometry 

In [29]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.52900788e+00  -1.96228033e+00  -8.24550000e-04]
1 C [ -1.87338440e-01  -1.91419006e+00  -1.07736000e-03]
2 C [ 0.34259171 -0.57038386 -0.00064109]
3 C [ -6.05111100e-01   3.80520380e-01  -3.01900000e-05]
4 S [ -2.25060957e+00  -3.24146360e-01   4.65300000e-05]
5 H [ -2.16232414e+00  -2.82023213e+00  -1.02345000e-03]
6 H [  4.31209940e-01  -2.78809225e+00  -1.53823000e-03]
7 H [  1.39112655e+00  -3.53910890e-01  -8.21500000e-04]
8 H [ -4.82375920e-01   1.43982288e+00   3.51820000e-04]


Store the results in a tar ball in the storage directory 

In [30]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

Again let's make it a function

In [31]:
def nw_esp(project_i,bb_i,res_i):
    '''Calculate ESP charges of a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_esp = streamm.NWChem('nw_esp_{}'.format(bb_i.tag))
    print(nwchem_esp.tag)
    # Add thiophene structure with optimized coordinates from previous calculation
    nwchem_esp.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_esp.set_resource(res_i)
    # Add calculation to project
    project_i.add_calc(nwchem_esp)
    # Make the local directories 
    nwchem_esp.make_dir()
    # Change to the `launch` directory
    os.chdir(nwchem_esp.dir['launch'])
    #
    nwchem_esp.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_esp.cp_file('templates','nw',"nwchem_esp.nw",'templates','launch')
    #
    nwchem_esp.load_str('templates','nw')        
    nwchem_esp.load_str('templates','run')
    # 
    nwchem_esp.properties['basis'] = '6-31g'
    nwchem_esp.properties['method'] = 'UHF'
    nwchem_esp.properties['charge'] = 0
    nwchem_esp.properties['spin_mult'] = 1
    nwchem_esp.properties['task'] = 'SCF'
    nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

    pprint(nwchem_esp.properties)

    nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

    nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
    nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

    nwchem_esp.add_file('output','log',"%s.log"%(nwchem_esp.tag))

    # Save details in .json files 

    os.chdir(nwchem_esp.dir['home'])
    project_i.dump_json()

    os.chdir(nwchem_esp.dir['launch'])
    nwchem_esp.push()
    nwchem_esp.run()
    # Add calculation to project
    project_i.add_calc(nwchem_esp,deepcopy = True)
    # 
    return project_i 
    
    

In [32]:
p3ht_et = nw_esp(p3ht_et,bbTh,peregrine)

nw_esp_thiophene
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.52900788      -1.96228033      -0.00082455 \n     C      -0.18733844      -1.91419006      -0.00107736 \n     C       0.34259171      -0.57038386      -0.00064109 \n     C      -0.60511110       0.38052038      -0.00003019 \n     S      -2.25060957      -0.32414636       0.00004653 \n     H      -2.16232414      -2.82023213      -0.00102345 \n     H       0.43120994      -2.78809225      -0.00153823 \n     H       1.39112655      -0.35391089      -0.00082150 \n     H      -0.48237592       1.43982288       0.00035182 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_esp_thiophene/'

Check status until finished

In [229]:
nwchem_i = p3ht_et.calculations['nw_esp_thiophene']

In [34]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)    

In [35]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_esp_thiophene has status finished


In [36]:
nwchem_i.analysis()

In [230]:
total_charge = 0.0 
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge
    total_charge += p.charge
print total_charge

0 C -0.085722
1 C -0.136866
2 C -0.085772
3 C -0.108841
4 S -0.11776
5 H 0.153759
6 H 0.119036
7 H 0.109131
8 H 0.153035
0.0


Update the charges of the Buildingblock

In [239]:
bbTh.tag += '_HFesp'

In [231]:
for pk,p in bbTh.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

0 C -0.085722
1 C -0.136866
2 C -0.085772
3 C -0.108841
4 S -0.11776
5 H 0.153759
6 H 0.119036
7 H 0.109131
8 H 0.153035


Store the results 

In [232]:
nwchem_i.store()

In [233]:
bbTh.bonded_nblist = bbTh.guess_nblist(0,radii_buffer=1.35)

In [234]:
bbTh.bonded_bonds()
bbTh.bonded_angles()
bbTh.bonded_dih()

Store a pickled object of the Buildingblock

In [42]:
os.chdir(res_local.dir['materials']) 
bbTh.dump_pickle()

Let us optimize the structure with the oplsaa force-field to check the parameters 

In [95]:
os.chdir(res_local.dir['home']) 

In [96]:
from pathlib2 import Path

In [97]:
need_files = ['oplsaa.pkl']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run forcefields.ipynb")
        os.system("jupyter nbconvert --to python  forcefields.ipynb")
        os.system("python forcefields.py")

In [98]:
oplsaa = streamm.forcefields.parameters.read_pickle('oplsaa')

In [99]:
print oplsaa.unit_conf['energy']

kCalmol


We need to add the conjugated carbons, hydrogen and sulfur atom types 

In [100]:
import streamm.forcefields.particletype as particletype

In [101]:
import pymatgen_core.core.periodic_table as periodic_table

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [102]:
CA = particletype.Particletype('CA')
HA = particletype.Particletype('HA')

In [103]:
CA.update_units(oplsaa.unit_conf)
HA.update_units(oplsaa.unit_conf)

In [104]:
CA.epsilon = 0.070 # kcal/mol
CA.sigma = 3.55 # Angstroms 

In [105]:
HA.epsilon = 0.030 # kcal/mol
HA.sigma = 2.42 # Angstroms 

In [106]:
CA.mass =  periodic_table.Element['C'].atomic_mass.real
HA.mass =  periodic_table.Element['H'].atomic_mass.real

In [107]:
print CA,HA

 CA epsilon:0.07 sigma:3.55  HA epsilon:0.03 sigma:2.42


In [108]:
S = particletype.Particletype('S')

In [109]:
S.update_units(oplsaa.unit_conf)

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [110]:
S.epsilon = 0.25 # kcal/mol
S.sigma = 3.55 # Angstroms 

In [111]:
S.mass =  periodic_table.Element['S'].atomic_mass.real

Add to forcefield parameters container

In [112]:
oplsaa.add_particletype(CA)
oplsaa.add_particletype(HA)
oplsaa.add_particletype(S)

Set the bond stretching parameters 

In [113]:
import streamm.forcefields.bondtype as bondtype

In [114]:
bt_i = bondtype.Bondtype('CA','HA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.080,367.0)
oplsaa.add_bondtype(bt_i)

In [115]:
bt_i = bondtype.Bondtype('CA','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.400,469.0)
oplsaa.add_bondtype(bt_i)

In [116]:
bt_i = bondtype.Bondtype('S','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.71,250.0)
oplsaa.add_bondtype(bt_i)

In [117]:
for btk,bt in oplsaa.bondtypes.iteritems():
    print btk,bt

0  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  
1  bond  CT - CT type harmonic 
  harmonic r_0 = 1.530000 K = 268.000000 lammps index 0  gromacs index 0  
2  bond  CA - HA type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  
3  bond  CA - CA type harmonic 
  harmonic r_0 = 1.400000 K = 469.000000 lammps index 0  gromacs index 0  
4  bond  S - CA type harmonic 
  harmonic r_0 = 1.710000 K = 250.000000 lammps index 0  gromacs index 0  


In [118]:
import streamm.forcefields.angletype as angletype

In [119]:
bat_i = angletype.Angletype('CA','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,63.0)
oplsaa.add_angletype(bat_i)

In [120]:
bat_i = angletype.Angletype('CA','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [121]:
bat_i = angletype.Angletype('CA','S','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(92.2,70.0)
oplsaa.add_angletype(bat_i)

In [122]:
bat_i = angletype.Angletype('S','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [123]:
bat_i = angletype.Angletype('S','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(111.0,70.0)
oplsaa.add_angletype(bat_i)

In [124]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

0  angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
1  angle  HC - CT - CT type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
2  angle  CA - CA - CA type harmonic 
  harmonic theta_0 = 120.000000 K = 63.000000 lammps index 0  gromacs index 0  
3  angle  CA - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
4  angle  CA - S - CA type harmonic 
  harmonic theta_0 = 92.200000 K = 70.000000 lammps index 0  gromacs index 0  
5  angle  S - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
6  angle  S - CA - CA type harmonic 
  harmonic theta_0 = 111.000000 K = 70.000000 lammps index 0  gromacs index 0  


Set some reasonable dihedral parameters

In [125]:
import streamm.forcefields.dihtype as dihtype

In [126]:
# oplsaa.dihtypes = {}

In [127]:
dih_i = dihtype.Dihtype('X','CA','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [128]:
dih_i = dihtype.Dihtype('X','S','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,2.416710,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [129]:
dih_i = dihtype.Dihtype('S','CA','CA','HA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [130]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

0  dihedral  HC - CT - CT - HC type opls 
  k1 = 0.000000 k2 = 0.000000 k3 = 0.300000 k4 = 0.000000 lammps index 0  gromcas index 0 
1  dihedral  X - CA - CA - X type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
2  dihedral  X - S - CA - X type opls 
  k1 = 0.000000 k2 = 2.416710 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
3  dihedral  S - CA - CA - HA type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 


Let us make an MD simulation of just the monomer to check that our parameters are okay

In [131]:
def lmp_run(project_i,bb_i,param_i,res_i,md_type = 'min'):
    # Create LAMMPS calculation object 
    lmp_i = streamm.LAMMPS('lmp_{}_{}'.format(md_type,bb_i.tag))
    # Set parameter container 
    lmp_i.paramC = param_i
    lmp_i.set_strucC(bb_i)
    # Set force-field parameters 
    lmp_i.set_ffparam()
    # Set resource to local
    lmp_i.set_resource(res_i)
    # Make local directories
    lmp_i.make_dir()
    # Set pbc's to on
    lmp_i.strucC.lat.pbcs = [True,True,True]
    # Change to launch directory
    os.chdir(lmp_i.dir['launch'])
    # Copy over the templates from the template directory 
    lmp_i.cp_file('templates','in',"lammps_{}.in".format(md_type),'templates','launch')
    lmp_i.cp_file('templates','run',"lammps_peregrine.pbs",'templates','launch')
    # Change to scratch
    os.chdir(lmp_i.dir['launch'])
    # Read in template files and store them as strings in the `str` dictionary
    lmp_i.load_str('templates','in')
    lmp_i.load_str('templates','run')
    # Write LAMMPS .data file
    lmp_i.write_data()
    # Replace keys in template string with properties 
    lmp_i.replacewrite_prop('in','input','in','%s.in'%(lmp_i.tag))
    # Add the input file to the properties to be written into the run file
    lmp_i.properties['input_in'] = lmp_i.files['input']['in']
    lmp_i.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_i.tag))
    # Save json file in root directory
    os.chdir(lmp_i.dir['home'])
    lmp_i.dump_json()
    # Run bash script or submit to cluster
    lmp_i.add_file('output','log',"%s.log"%(lmp_i.tag))
    # Save details in .json files 
    os.chdir(lmp_i.dir['home'])
    project_i.dump_json()
    #
    os.chdir(lmp_i.dir['launch'])
    lmp_i.push()
    lmp_i.run()
    # Add calculation to project
    project_i.add_calc(lmp_i,deepcopy = True)
    # 
    return project_i     

In [132]:
p3ht_et = lmp_run(p3ht_et,bbTh,oplsaa,peregrine)

In [134]:
lmp_i = p3ht_et.calculations['lmp_min_thiophene']

In [135]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [137]:
lmp_i.analysis()

>  using mdrun with len 1 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '15.562316', '15.562316', '-0.42232458', '1000000', '100', '100', '100', '7.4306668', '3.7498641', '4.9478895e-08', '0', '-0.0096913433', '6.497175', '-2.1056982', '0']
1 9823u  [0.0]
> col  1 ['8', '0', '7.7379031', '7.7379031', '0.014732998', '1000000', '100', '100', '100', '0.1170396', '3.2551899', '2.7812433e-08', '0', '-0.0095403847', '6.4805932', '-2.1053793', '0']
2 9823u  [0.0, 8.0]
 Calc 0 finished 


Energy decreased and nothing exploded so that's good

In [138]:
lmp_i.store()

Read in data file positions

In [139]:
lmp_i.pull()

Read in data file output and update positions

In [140]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_min_thiophene/


In [141]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [142]:
lmp_i.read_data_pos(datafn)

0 [-1.5684056139810723, -1.9370746707518094, -0.00079793956581835]
1 [-0.17270368216398252, -1.8919357186900865, -0.0010708242700525178]
2 [0.338054513246081, -0.5966486617476253, -0.0006531518007889394]
3 [-0.651128019207314, 0.38904587521034983, -1.9444636718466806e-05]
4 [-2.2118400697153504, -0.3394197074796044, 2.7358399738847222e-05]
5 [-2.1603308845727027, -2.8420307382421717, -0.0010310960009818034]
6 [0.44633138319251936, -2.7765945425475413, -0.001543139842146459]
7 [1.3942731048991792, -0.37271830949730206, -0.0008235860329267993]
8 [-0.4660887316973578, 1.4544844737457918, 0.0003558237496944879]


In [143]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [144]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [240]:
bbTh.tag += '_oplsaa'

In [146]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.56840561e+00  -1.93707467e+00  -7.97939566e-04]
1 C [ -1.72703682e-01  -1.89193572e+00  -1.07082427e-03]
2 C [ 0.33805451 -0.59664866 -0.00065315]
3 C [ -6.51128019e-01   3.89045875e-01  -1.94446367e-05]
4 S [ -2.21184007e+00  -3.39419707e-01   2.73583997e-05]
5 H [ -2.16033088e+00  -2.84203074e+00  -1.03109600e-03]
6 H [  4.46331383e-01  -2.77659454e+00  -1.54313984e-03]
7 H [  1.39427310e+00  -3.72718309e-01  -8.23586033e-04]
8 H [ -4.66088732e-01   1.45448447e+00   3.55823750e-04]


Save the Buildingblock and force-field

In [152]:
os.chdir(res_local.dir['materials']) 
bbTh.write_xyz()
# bbTh.dump_pickle() # Not working for 
oplsaa.dump_pickle()

Okay now that we have a handle on thiophene let's follow the same procedure for hexane

Build hexane

In [206]:
bbHex = streamm.Buildingblock('hexane')
symbols = ['C','H','H','H','C','H','H','C','H','H','C','H','H','C','H','H','C','H','H','H']
positions = [ ]
positions.append([-6.410969,-0.381641,-0.000031])
positions.append([-7.310084,0.245311,-0.000038])
positions.append([-6.456117,-1.028799,0.884636])
positions.append([-6.456111,-1.028812,-0.884689])
positions.append([-5.135268,0.467175,-0.000033])
positions.append([-5.135484,1.128782,0.877977])
positions.append([-5.135479,1.128771,-0.87805])
positions.append([-3.850566,-0.371258,-0.000024])
positions.append([-3.85112,-1.033978,0.87841])
positions.append([-3.851114,-1.033987,-0.878451])
positions.append([-2.567451,0.469603,-0.000024])
positions.append([-2.567784,1.132155,0.8784])
positions.append([-2.567776,1.132146,-0.878455])
positions.append([-1.283527,-0.370234,-0.000013])
positions.append([-1.28337,-1.032804,0.87836])
positions.append([-1.28336,-1.032812,-0.87838])
positions.append([0.00482234,0.47342231,-0.00000898])
positions.append([0.02595107,1.09220686,0.87266464])
positions.append([0.85585781,-0.17514133,0.00194589])
positions.append([0.02780957,1.08937798,-0.87463473])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbHex.add_partpos(pt_i,pos_i)

In [207]:
bbHex.particles[0].rsite = 'rg'
bbHex.particles[1].rsite = 'rgcap'

In [208]:
c_cnt =1
h_cnt =1
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.resname = "SCP3"
                particle_i.residue = c_cnt
                c_cnt +=1 
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.resname = "HC"
                particle_i.residue = c_cnt -1 
                h_cnt +=1 

Set the parameter keys and some reasonable atomic charges 

In [209]:
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CT'
                particle_i.charge = -0.12

            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HC'
                particle_i.charge = 0.06
            print pkey_i, particle_i.symbol,particle_i.charge

0 C -0.12
1 H 0.06
2 H 0.06
3 H 0.06
4 C -0.12
5 H 0.06
6 H 0.06
7 C -0.12
8 H 0.06
9 H 0.06
10 C -0.12
11 H 0.06
12 H 0.06
13 C -0.12
14 H 0.06
15 H 0.06
16 C -0.12
17 H 0.06
18 H 0.06
19 H 0.06


In [210]:
bbHex.particles[0].charge  = -0.18
bbHex.particles[16].charge  = -0.18

Check that the molecule is neutral 

In [211]:
bbHex.calc_charge()
print bbHex.charge


0.0


Now let us optimze and calculate ESP charges for hexane

Optimize structure with NWChem

In [160]:
p3ht_et = nw_opt(p3ht_et,bbHex,peregrine)

nw_opt_hexane
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -6.41096900      -0.38164100      -0.00003100 \n     H      -7.31008400       0.24531100      -0.00003800 \n     H      -6.45611700      -1.02879900       0.88463600 \n     H      -6.45611100      -1.02881200      -0.88468900 \n     C      -5.13526800       0.46717500      -0.00003300 \n     H      -5.13548400       1.12878200       0.87797700 \n     H      -5.13547900       1.12877100      -0.87805000 \n     C      -3.85056600      -0.37125800      -0.00002400 \n     H      -3.85112000      -1.03397800       0.87841000 \n     H      -3.85111400      -1.03398700      -0.87845100 \n     C      -2.56745100       0.46960300      -0.00002400 \n     H      -2.56778400       1.13215500       0.87840000 \n     H      -2.56777600       1.13214600      -0.87845500 \n     C      -1.28352700      -0.37023400      -0.00001

In [161]:
nwchem_i = p3ht_et.calculations['nw_opt_hexane']

In [162]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)

In [163]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status finished
Calculation nw_esp_thiophene has status stored
Calculation lmp_min_thiophene has status stored


Get the calculation from the project object 

In [164]:
nwchem_i.analysis()

Print energies 

In [165]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.5549424, -0.5282407, -0.5218841, -0.4996812, -0.4774207, -0.4626707, -0.453462, -0.4506618, -0.4351213, 0.2272358]
-235.271544596


Check that the positions of the structure have been optimized 

In [166]:
for pk,p in bbHex.particles.iteritems():
    print pk,p.symbol,bbHex.positions[pk]

0 C [ -6.41096900e+00  -3.81641000e-01  -3.10000000e-05]
1 H [ -7.31008400e+00   2.45311000e-01  -3.80000000e-05]
2 H [-6.456117 -1.028799  0.884636]
3 H [-6.456111 -1.028812 -0.884689]
4 C [ -5.13526800e+00   4.67175000e-01  -3.30000000e-05]
5 H [-5.135484  1.128782  0.877977]
6 H [-5.135479  1.128771 -0.87805 ]
7 C [ -3.85056600e+00  -3.71258000e-01  -2.40000000e-05]
8 H [-3.85112  -1.033978  0.87841 ]
9 H [-3.851114 -1.033987 -0.878451]
10 C [ -2.56745100e+00   4.69603000e-01  -2.40000000e-05]
11 H [-2.567784  1.132155  0.8784  ]
12 H [-2.567776  1.132146 -0.878455]
13 C [ -1.28352700e+00  -3.70234000e-01  -1.30000000e-05]
14 H [-1.28337  -1.032804  0.87836 ]
15 H [-1.28336  -1.032812 -0.87838 ]
16 C [  4.82234000e-03   4.73422310e-01  -8.98000000e-06]
17 H [ 0.02595107  1.09220686  0.87266464]
18 H [ 0.85585781 -0.17514133  0.00194589]
19 H [ 0.02780957  1.08937798 -0.87463473]


In [167]:
print nwchem_i.strucC.positions

[[ -6.40727097e+00  -3.79022830e-01   3.48200000e-05]
 [ -7.29437061e+00   2.44910680e-01   7.42600000e-05]
 [ -6.45166131e+00  -1.01740717e+00   8.76562170e-01]
 [ -6.45174259e+00  -1.01732202e+00  -8.76555730e-01]
 [ -5.13169158e+00   4.68167590e-01   1.40600000e-05]
 [ -5.13055154e+00   1.11968847e+00   8.70404090e-01]
 [ -5.13070050e+00   1.11990885e+00  -8.70210450e-01]
 [ -3.85233214e+00  -3.76683220e-01  -2.30580000e-04]
 [ -3.85428614e+00  -1.02883087e+00   8.70880990e-01]
 [ -3.85451310e+00  -1.02885460e+00  -8.71281430e-01]
 [ -2.57107059e+00   4.64329490e-01  -2.23210000e-04]
 [ -2.56902452e+00   1.11699822e+00   8.70459640e-01]
 [ -2.56884865e+00   1.11635494e+00  -8.71476110e-01]
 [ -1.29180042e+00  -3.80715840e-01   8.83000000e-06]
 [ -1.29320068e+00  -1.03207065e+00   8.70577820e-01]
 [ -1.29288442e+00  -1.03276931e+00  -8.69939080e-01]
 [ -1.54094400e-02   4.65275360e-01   1.41120000e-04]
 [  2.90437300e-02   1.10462399e+00   8.76019630e-01]
 [  8.70896710e-01  -1.59878

Update positions in Buildingblock object

In [168]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

0 C [ -6.40727097e+00  -3.79022830e-01   3.48200000e-05]
1 H [ -7.29437061e+00   2.44910680e-01   7.42600000e-05]
2 H [-6.45166131 -1.01740717  0.87656217]
3 H [-6.45174259 -1.01732202 -0.87655573]
4 C [ -5.13169158e+00   4.68167590e-01   1.40600000e-05]
5 H [-5.13055154  1.11968847  0.87040409]
6 H [-5.1307005   1.11990885 -0.87021045]
7 C [ -3.85233214e+00  -3.76683220e-01  -2.30580000e-04]
8 H [-3.85428614 -1.02883087  0.87088099]
9 H [-3.8545131  -1.0288546  -0.87128143]
10 C [ -2.57107059e+00   4.64329490e-01  -2.23210000e-04]
11 H [-2.56902452  1.11699822  0.87045964]
12 H [-2.56884865  1.11635494 -0.87147611]
13 C [ -1.29180042e+00  -3.80715840e-01   8.83000000e-06]
14 H [-1.29320068 -1.03207065  0.87057782]
15 H [-1.29288442 -1.03276931 -0.86993908]
16 C [ -1.54094400e-02   4.65275360e-01   1.41120000e-04]
17 H [ 0.02904373  1.10462399  0.87601963]
18 H [ 0.87089671 -0.15987895  0.00136176]
19 H [ 0.03027954  1.10278169 -0.87706076]


Store the results in a tar ball in the storage directory 

In [169]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

In [170]:
p3ht_et = nw_esp(p3ht_et,bbHex,peregrine)

nw_esp_hexane
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -6.40727097      -0.37902283       0.00003482 \n     H      -7.29437061       0.24491068       0.00007426 \n     H      -6.45166131      -1.01740717       0.87656217 \n     H      -6.45174259      -1.01732202      -0.87655573 \n     C      -5.13169158       0.46816759       0.00001406 \n     H      -5.13055154       1.11968847       0.87040409 \n     H      -5.13070050       1.11990885      -0.87021045 \n     C      -3.85233214      -0.37668322      -0.00023058 \n     H      -3.85428614      -1.02883087       0.87088099 \n     H      -3.85451310      -1.02885460      -0.87128143 \n     C      -2.57107059       0.46432949      -0.00022321 \n     H      -2.56902452       1.11699822       0.87045964 \n     H      -2.56884865       1.11635494      -0.87147611 \n     C      -1.29180042      -0.38071584       0.00000

Check status unit finished

In [212]:
nwchem_i = p3ht_et.calculations['nw_esp_hexane']

In [172]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)

In [173]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation nw_esp_thiophene has status stored
Calculation lmp_min_thiophene has status stored
Calculation nw_esp_hexane has status finished


In [174]:
nwchem_i.analysis()

In [213]:
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

0 C -0.105262
1 H 0.014539
2 H 0.012048
3 H 0.013508
4 C 0.19358
5 H -0.055571
6 H -0.050691
7 C 0.02063
8 H -0.02177
9 H -0.023452
10 C 0.038524
11 H -0.031295
12 H -0.028867
13 C 0.205973
14 H -0.055658
15 H -0.059938
16 C -0.09414
17 H 0.006364
18 H 0.00836
19 H 0.013116


In [214]:
nwchem_i.strucC.calc_charge()
print nwchem_i.strucC.charge

-2.00000000003e-06


Print energies 

In [215]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.5549626, -0.5258815, -0.5254663, -0.5002771, -0.4784797, -0.4652444, -0.4549941, -0.4542301, -0.4337884, 0.2280542]
-235.271544554


Update the charges of the Buildingblock

In [216]:
for pk,p in bbHex.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge

In [217]:
bbHex.tag += '_HFesp'

Store the results 

In [180]:
nwchem_i.store()

First we need to identify the bonding within the Buildingblock

In [235]:
bbHex.bonded_nblist = bbHex.guess_nblist(0,radii_buffer=1.35)

In [236]:
bbHex.bonded_bonds()
bbHex.bonded_angles()
bbHex.bonded_dih()

Add the need parameters the the oplsaa parameter container

In [184]:
bat_i = angletype.Angletype('CT','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,40.0)
oplsaa.add_angletype(bat_i)

In [184]:
bat_i = angletype.Angletype('CT','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,40.0)
oplsaa.add_angletype(bat_i)

In [186]:
bat_i = angletype.Angletype('CT','CT','HC',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,50.0)
oplsaa.add_angletype(bat_i)

In [188]:
dih_i = dihtype.Dihtype('CT','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.433341,-0.016667,0.066668,0.0)
oplsaa.add_dihtype(dih_i)

In [189]:
dih_i = dihtype.Dihtype('HC','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [190]:
dih_i = dihtype.Dihtype('HC','CT','CT','HC',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

Run a oplsaa minimization to get the minimized structure

In [191]:
p3ht_et = lmp_run(p3ht_et,bbHex,oplsaa,peregrine)

In [193]:
p3ht_et.check()

Calculation lmp_min_hexane_HFesp has status written
Calculation nw_esp_thiophene has status stored
Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation lmp_min_thiophene has status stored
Calculation nw_esp_hexane has status stored


In [220]:
lmp_i = p3ht_et.calculations['lmp_min_hexane_HFesp']

In [195]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [196]:
lmp_i.analysis()

>  using mdrun with len 1 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '0.41190105', '0.41190105', '-1.9261181', '1000000', '100', '100', '100', '0.23947518', '2.3669071', '0.013163119', '0', '-0.77470675', '-0.33289267', '-1.100045', '0']
1 9823u  [0.0]
> col  1 ['25', '0', '-2.0815576', '-2.0815576', '-0.086547071', '1000000', '100', '100', '100', '0.0033958522', '0.09254902', '0.0004568671', '0', '-0.7104337', '-0.36748046', '-1.1000452', '0']
2 9823u  [0.0, 25.0]
 Calc 0 finished 


Energy decreased and nothing exploded so that's good

In [197]:
lmp_i.store()

Read in data file positions

In [198]:
lmp_i.pull()

Read in data file output and update positions

In [199]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_min_hexane_HFesp/


In [200]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [221]:
lmp_i.read_data_pos(datafn)

0 [-6.342407538882333, -0.3823535845037426, -5.56491658941105e-07]
1 [-7.212078485772054, 0.25818355236204, -3.3285498846693436e-05]
2 [-6.343125726868117, -1.0018577388750032, 0.8843975173198843]
3 [-6.343049925595568, -1.0017695015312, -0.884579733057865]
4 [-5.08525165042471, 0.4916046657856369, -1.2412219040184932e-05]
5 [-5.084059921212749, 1.1087690988645975, 0.8861658653625354]
6 [-5.084033401934145, 1.109002111958136, -0.8857964992163528]
7 [-3.8381921274040636, -0.39680888852009005, 1.2673254194045647e-05]
8 [-3.8404980690805255, -1.013802362263817, 0.8861211285207612]
9 [-3.8409079009633063, -1.013937924608017, -0.8863178523993515]
10 [-2.5845900611304917, 0.48503750530838796, -0.0005111260343248514]
11 [-2.5816037728604107, 1.1024584708362954, 0.8856383668297363]
12 [-2.581507018913794, 1.1015401650443017, -0.8869735160267269]
13 [-1.338431378355753, -0.4044830160914846, 0.000290122405832904]
14 [-1.3404732068959135, -1.0214259707727342, 0.8863687940568565]
15 [-1.3400891802

In [202]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [203]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [218]:
bbHex.tag += '_oplsaa'

In [223]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

 0 C [ -6.34240754e+00  -3.82353585e-01  -5.56491659e-07]
1 H [ -7.21207849e+00   2.58183552e-01  -3.32854988e-05]
2 H [-6.34312573 -1.00185774  0.88439752]
3 H [-6.34304993 -1.0017695  -0.88457973]
4 C [ -5.08525165e+00   4.91604666e-01  -1.24122190e-05]
5 H [-5.08405992  1.1087691   0.88616587]
6 H [-5.0840334   1.10900211 -0.8857965 ]
7 C [ -3.83819213e+00  -3.96808889e-01   1.26732542e-05]
8 H [-3.84049807 -1.01380236  0.88612113]
9 H [-3.8409079  -1.01393792 -0.88631785]
10 C [ -2.58459006e+00   4.85037505e-01  -5.11126034e-04]
11 H [-2.58160377  1.10245847  0.88563837]
12 H [-2.58150702  1.10154017 -0.88697352]
13 C [ -1.33843138e+00  -4.04483016e-01   2.90122406e-04]
14 H [-1.34047321 -1.02142597  0.88636879]
15 H [-1.34008918 -1.02209928 -0.88559241]
16 C [ -8.05783703e-02   4.68424646e-01   6.94717265e-05]
17 H [-0.07976047  1.08855879  0.88419035]
18 H [ 0.78861022 -0.17281355  0.00093658]
19 H [-0.07911102  1.0872558  -0.88481048]


Save the Buildingblock and force-field

In [237]:
os.chdir(res_local.dir['materials']) 
bbHex.write_xyz()
# bbTh.dump_pickle() # Not working for 
oplsaa.dump_pickle()

In [241]:
print bbHex.tag,bbTh.tag

hexane_HFesp_oplsaa thiophene_HFesp_oplsaa


So let us make some P3HT oligomers 

In [243]:
os.chdir(res_local.dir['materials']) 

In [244]:
bbTh.find_rsites()
bbHex.find_rsites()

In [245]:
print(bbTh.show_rsites())

rsite:termcap[ paticle:atom[5] H1 (H) index:5 n_bonds:1] 
rsite:termcap[ paticle:atom[8] H4 (H) index:8 n_bonds:1] 
rsite:funccap[ paticle:atom[6] H2 (H) index:6 n_bonds:1] 



In [246]:
print(bbHex.show_rsites())

rsite:rg[ paticle:atom[0] C1 (C) index:0 n_bonds:4] 
rsite:rgcap[ paticle:atom[1] H1 (H) index:1 n_bonds:1] 



In [247]:
import streamm.structures.buildingblock as bb

In [248]:
ht = bb.attach(bbTh,bbHex,'funccap',0,'rgcap',0,tag='3-hexyl-thiophene')

Update bond angles and dihedrals after Buildingblock join

In [284]:
ht.bonded_bonds()
ht.bonded_angles()
ht.bonded_dih()

Check that the molecule looks good

In [285]:
ht.write_xyz()

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [286]:
ht.calc_charge()
ht.charge

-2.000000000033225e-06

In [287]:
print(ht.show_rsites())

rsite:termcap[ paticle:atom[5] H1 (H) index:5 n_bonds:1] 
rsite:termcap[ paticle:atom[7] H4 (H) index:7 n_bonds:1] 
rsite:rg[ paticle:atom[8] C1 (C) index:8 n_bonds:4] 



Add inter thiophene hexane parameters

In [288]:
bt_i = bondtype.Bondtype('CT','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.51,317.0)
oplsaa.add_bondtype(bt_i)

Bond angle parameters 

In [289]:
bat_i = angletype.Angletype('CA','CA','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,70.0)
oplsaa.add_angletype(bat_i)


bat_i = angletype.Angletype('HA','CA','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)



bat_i = angletype.Angletype('CA','CT','HC',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.5,50.0)
oplsaa.add_angletype(bat_i)

bat_i = angletype.Angletype('CA','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(114.0,63.0)
oplsaa.add_angletype(bat_i)

In [290]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

0  angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
1  angle  HC - CT - CT type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
2  angle  CA - CA - CA type harmonic 
  harmonic theta_0 = 120.000000 K = 63.000000 lammps index 0  gromacs index 0  
3  angle  CA - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
4  angle  CA - S - CA type harmonic 
  harmonic theta_0 = 92.200000 K = 70.000000 lammps index 0  gromacs index 0  
5  angle  S - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
6  angle  S - CA - CA type harmonic 
  harmonic theta_0 = 111.000000 K = 70.000000 lammps index 0  gromacs index 0  
7  angle  CT - CT - CT type harmonic 
  harmonic theta_0 = 109.500000 K = 40.000000 lammps index 0  gromacs index 0  
8  angle  CT - CT - HC type harmonic 
  harmonic theta_0 = 1

Note: The inter-ring torsional is not consider as a seperate set of parameters for the simplicity of this example

In [293]:
dih_i = dihtype.Dihtype('HC','CT','CT','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [302]:
dih_i = dihtype.Dihtype('CT','CT','CT','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.433341,-0.016667,0.066668,0.0)
oplsaa.add_dihtype(dih_i)

In [296]:
dih_i = dihtype.Dihtype('HC','CT','CA','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [299]:
dih_i = dihtype.Dihtype('CT','CT','CA','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [300]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

0  dihedral  HC - CT - CT - HC type opls 
  k1 = 0.000000 k2 = 0.000000 k3 = 0.300000 k4 = 0.000000 lammps index 0  gromcas index 0 
1  dihedral  X - CA - CA - X type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
2  dihedral  X - S - CA - X type opls 
  k1 = 0.000000 k2 = 2.416710 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
3  dihedral  S - CA - CA - HA type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
4  dihedral  CT - CT - CT - CT type opls 
  k1 = 0.433341 k2 = -0.016667 k3 = 0.066668 k4 = 0.000000 lammps index 0  gromcas index 0 
5  dihedral  HC - CT - CT - CT type opls 
  k1 = 0.000000 k2 = -0.000000 k3 = 0.100000 k4 = 0.000000 lammps index 0  gromcas index 0 
6  dihedral  HC - CT - CT - HC type opls 
  k1 = 0.000000 k2 = -0.000000 k3 = 0.100000 k4 = 0.000000 lammps index 0  gromcas index 0 
7  dihedral  HC - CT - CT - CA type opls 
  k1 = 0.000000 k2 = -0.000000

Run a oplsaa minimization to get the minimized structure

In [303]:
p3ht_et = lmp_run(p3ht_et,ht,oplsaa,peregrine)

Traceback (most recent call last):
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 861, in emit
    msg = self.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 734, in format
    return fmt.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 465, in format
    record.message = record.getMessage()
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 329, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Logged from file calculation.py, line 690
Traceback (most recent call last):
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 861, in emit
    msg = self.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 734, in format
    return fmt.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 465, in format
    record.mes

In [304]:
p3ht_et.check()

Calculation lmp_min_hexane_HFesp has status stored
Calculation nw_esp_thiophene has status stored
Calculation nw_opt_thiophene has status stored
Calculation lmp_min_3-hexyl-thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation lmp_min_thiophene has status stored
Calculation lmp_min_3-hexyl-thiophene_oplsaa has status written
Calculation nw_esp_hexane has status stored


In [306]:
lmp_i = p3ht_et.calculations['lmp_min_3-hexyl-thiophene']

In [307]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [308]:
lmp_i.analysis()

>  using mdrun with len 1 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '11.399494', '11.399494', '0.088311758', '1000000', '100', '100', '100', '0.098327522', '7.7759379', '0.20107781', '0', '-1.2093806', '7.3074506', '-2.7739196', '0']
1 9823u  [0.0]
> col  1 ['17', '0', '7.6328379', '7.6328379', '0.19633034', '1000000', '100', '100', '100', '0.11428404', '3.8989799', '0.20163858', '0', '-1.057215', '7.249052', '-2.7739016', '0']
2 9823u  [0.0, 17.0]
 Calc 0 finished 


Energy decreased and nothing exploded so that's good

In [309]:
lmp_i.store()

Read in data file positions

In [310]:
lmp_i.pull()

Read in data file output and update positions

In [311]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_min_3-hexyl-thiophene_oplsaa/


In [312]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [313]:
lmp_i.read_data_pos(datafn)

24 [-5.527477461464044, -4.905371070931063, 0.8840462687789127]
26 [-5.527701949410191, -4.905021991320866, -0.884325136769373]
20 [-5.468402175642445, -2.955285552324124, 0.00014667141950947915]
21 [-5.842537081152703, -2.468131162723726, 0.8882061746086831]
22 [-5.843404136151737, -2.467737435596794, -0.8873293647600919]
23 [-5.9133043925219635, -4.419731882027037, 2.500712193639122e-06]
25 [-6.992147090629334, -4.457404031828984, 0.00015971875679166817]
0 [0.8726052457856165, 1.1712919786127232, -0.0007484007308327574]
1 [0.16554736267787, -0.033345806689422275, -0.0005557681407883796]
2 [0.9839274857692436, -1.1626688135179333, -9.236967873136725e-05]
3 [2.3443147082540396, -0.8506533791681496, -0.000165747427632165]
4 [2.565492415018743, 0.8577047132623278, -6.662977140384209e-05]
5 [0.42491658130005694, 2.1552830369908595, 0.00022848106302568387]
6 [0.6272372963230932, -2.182678155487773, 0.0012921625528223282]
7 [3.143298019522173, -1.577562826085836, 0.0010945955311302711]
8 [-

In [314]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [315]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [268]:
ht.tag += '_oplsaa'

In [269]:
for pk,p in ht.particles.iteritems():
    ht.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,ht.positions[pk]

0 C [  8.19506486e-01   1.10111983e+00  -6.86476607e-04]
1 C [ 0.13609175 -0.12016286 -0.00023107]
2 C [  1.00347430e+00  -1.21177477e+00  -1.20154076e-04]
3 C [  2.35396842e+00  -8.64526955e-01  -2.45335666e-04]
4 S [  2.52295837e+00   8.45769468e-01  -1.03410286e-05]
5 H [  3.50495491e-01   2.07469182e+00   2.87510191e-04]
6 H [  6.73288023e-01  -2.24006710e+00   1.25399957e-03]
7 H [  3.17179666e+00  -1.57145729e+00   1.04534620e-03]
8 C [ -1.37450557e+00  -1.68170109e-02  -4.32546771e-04]
9 H [-1.70780472  0.49995342  0.88735419]
10 H [-1.70863118  0.50000973 -0.88787442]
11 C [ -1.93124005e+00  -1.44428553e+00  -2.73010843e-04]
12 H [-1.58522339 -1.95539447  0.8861431 ]
13 H [-1.58556737 -1.95539379 -0.88682515]
14 C [ -3.45891385e+00  -1.44292112e+00  -5.71158329e-05]
15 H [-3.81835261 -0.94584429  0.88890749]
16 H [-3.81849807 -0.94513283 -0.88855736]
17 C [ -3.92879673e+00  -2.90059160e+00  -5.75947378e-04]
18 H [-3.55631034 -3.39457241  0.884721  ]
19 H [-3.55708709 -3.3935905

Save the Buildingblock and force-field

In [316]:
os.chdir(res_local.dir['materials']) 
ht.write_xyz()
# bbTh.dump_pickle() # Not working for 
oplsaa.dump_pickle()

Okay we have the monomer, so let's make a pentamer 

In [317]:
penta_ht = copy.deepcopy(ht)

In [318]:
# penta_ht = ht.prepattach('termcap',0,dir=-1,yangle=90.0)

In [319]:
for n in range(4):
    penta_ht = bb.attach(penta_ht,ht,'termcap',1,'termcap',0,tag='penta_3-hexyl-thiophene')

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [320]:
penta_ht.calc_charge()
penta_ht.charge

-9.999999999985715e-06

In [321]:
penta_ht.write_xyz()

Well it's cis, but we can run some high temperature MD to randomize that 

Update bond angles and dihedrals after Buildingblock join

In [322]:
penta_ht.bonded_bonds()
penta_ht.bonded_angles()
penta_ht.bonded_dih()

In [323]:
print penta_ht.print_properties()

 n_particles:127 
 n_bonds:131
 n_angles:245
 n_dihedrals:351
 n_impropers:0


Run a oplsaa minimization to get the minimized structure

In [324]:
p3ht_et = lmp_run(p3ht_et,penta_ht,oplsaa,peregrine)

Traceback (most recent call last):
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 861, in emit
    msg = self.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 734, in format
    return fmt.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 465, in format
    record.message = record.getMessage()
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 329, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Logged from file calculation.py, line 690
Traceback (most recent call last):
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 861, in emit
    msg = self.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 734, in format
    return fmt.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 465, in format
    record.mes

In [328]:
p3ht_et.check()

Calculation lmp_min_penta_3-hexyl-thiophene has status finished
Calculation lmp_min_hexane_HFesp has status stored
Calculation nw_esp_thiophene has status stored
Calculation nw_opt_thiophene has status stored
Calculation lmp_min_3-hexyl-thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation lmp_min_thiophene has status stored
Calculation lmp_min_3-hexyl-thiophene_oplsaa has status stored
Calculation nw_esp_hexane has status stored


In [330]:
lmp_i = p3ht_et.calculations['lmp_min_penta_3-hexyl-thiophene']

In [331]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    # time.sleep(30)

In [332]:
lmp_i.analysis()

>  using mdrun with len 1 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '669.25223', '669.25223', '179.23982', '1000000', '100', '100', '100', '7.2430833', '70.753731', '1.005452', '0', '591.08212', '12.390297', '-13.222452', '0']
1 9823u  [0.0]
> col  1 ['49', '0', '62.281426', '62.281426', '4.0178179', '1000000', '100', '100', '100', '5.1515074', '51.799809', '1.0411164', '0', '4.3197426', '13.193588', '-13.224337', '0']
2 9823u  [0.0, 49.0]
 Calc 0 finished 


Energy decreased and nothing exploded so that's good

In [333]:
lmp_i.store()

Read in data file positions

In [334]:
lmp_i.pull()

Read in data file output and update positions

In [335]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_min_penta_3-hexyl-thiophene/


In [336]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [337]:
lmp_i.read_data_pos(datafn)

0 [8.62838291109863, -9.614802497150665, 0.04147399462582152]
1 [9.879070384253055, -8.983268091610563, 0.04300988098344951]
2 [9.782647265958104, -7.6062029384983045, 0.03276909839876034]
3 [8.478961801433817, -7.124126850899608, 0.01405435988421265]
4 [7.35251949545295, -8.425177231484328, 0.03645673428526574]
5 [8.471126079944755, -10.683539101714368, 0.04846444563794985]
6 [10.648483817260049, -6.984411272587224, 0.042071778036145785]
7 [11.233957217645086, -9.676573991210867, 0.05528930197459048]
8 [11.298541660852099, -10.291534582663706, 0.9425062749725864]
9 [11.310256380978595, -10.299324885819921, -0.8255487295182599]
10 [12.425625238757181, -8.687811108439584, 0.05865442473946396]
11 [12.35666550551104, -8.067692291789518, 0.9399388493370662]
12 [12.367597174012172, -8.074779444538184, -0.8283580259673184]
26 [8.074630180336802, -5.772244649013335, 0.03833235606044619]
30 [6.383141368034369, -5.31918415221572, 0.016873050100964168]
32 [10.420342180858546, -4.593689034998712,

In [338]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [339]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [340]:
penta_ht.tag += '_oplsaa'

In [341]:
for pk,p in penta_ht.particles.iteritems():
    penta_ht.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,penta_ht.positions[pk]

0 C [ 8.62838291 -9.6148025   0.04147399]
1 C [ 9.87907038 -8.98326809  0.04300988]
2 C [ 9.78264727 -7.60620294  0.0327691 ]
3 C [ 8.4789618  -7.12412685  0.01405436]
4 S [ 7.3525195  -8.42517723  0.03645673]
5 H [  8.47112608 -10.6835391    0.04846445]
6 H [ 10.64848382  -6.98441127   0.04207178]
7 C [ 11.23395722  -9.67657399   0.0552893 ]
8 H [ 11.29854166 -10.29153458   0.94250627]
9 H [ 11.31025638 -10.29932489  -0.82554873]
10 C [ 12.42562524  -8.68781111   0.05865442]
11 H [ 12.35666551  -8.06769229   0.93993885]
12 H [ 12.36759717  -8.07477944  -0.82835803]
13 C [ 13.80369793  -9.34751239   0.06959028]
14 H [ 13.90703396  -9.94453685   0.96238491]
15 H [ 13.91735878  -9.95216696  -0.81680095]
16 C [ 14.86270025  -8.23344551   0.07096294]
17 H [ 14.74205778  -7.62010683   0.95261903]
18 H [ 14.75233548  -7.62782282  -0.81735204]
19 C [ 16.26111663  -8.85472014   0.08184265]
20 H [ 16.37286821  -9.45680317   0.97116133]
21 H [ 16.3835382   -9.46370136  -0.80135214]
22 C [ 17.334

Save the Buildingblock and force-field

In [342]:
os.chdir(res_local.dir['materials']) 
penta_ht.write_xyz()
# bbTh.dump_pickle() # Not working for 
oplsaa.dump_pickle()

Cool let's run some MD

In [343]:
p3ht_et = lmp_run(p3ht_et,penta_ht,oplsaa,peregrine,md_type='nvt')

In [344]:
p3ht_et.check()

Calculation lmp_min_penta_3-hexyl-thiophene has status stored
Calculation lmp_min_hexane_HFesp has status stored
Calculation nw_esp_thiophene has status stored
Calculation nw_opt_thiophene has status stored
Calculation lmp_min_3-hexyl-thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation lmp_min_thiophene has status stored
Calculation lmp_min_3-hexyl-thiophene_oplsaa has status stored
Calculation lmp_nvt_penta_3-hexyl-thiophene_oplsaa has status written
Calculation nw_esp_hexane has status stored


In [326]:
lmp_i = p3ht_et.calculations['lmp_min_3-hexyl-thiophene']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [ ]:
lmp_i.analysis()

Sweet as bro!